In [1]:
import numpy as np
import pandas as pd
import gc
import time

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm

import warnings
from time import time
import xgboost as xgb
xgb.__version__

'1.6.0rc1'

In [2]:
%%time
train = pd.read_csv('../../TPS_2021/input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../../TPS_2021/input/tabular-playground-series-dec-2021/test.csv')

CPU times: user 5.97 s, sys: 816 ms, total: 6.78 s
Wall time: 6.78 s


In [3]:
train.drop(columns=['Soil_Type7', 'Soil_Type15'], inplace=True) 
test.drop(columns=['Soil_Type7', 'Soil_Type15'], inplace=True)

In [4]:
train.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,0,3189,40,8,30,13,3270,206,234,193,...,0,0,0,0,0,0,0,0,0,1
1,1,3026,182,5,280,29,3270,233,240,106,...,0,0,0,0,0,0,0,0,0,2
2,2,3106,13,7,351,37,2914,208,234,137,...,0,0,0,0,0,0,0,0,0,1
3,3,3022,276,13,192,16,3034,207,238,156,...,0,0,0,0,0,0,0,0,0,2
4,4,2906,186,13,266,22,2916,231,231,154,...,0,0,0,0,0,0,0,0,0,2


In [5]:
features = train.columns[1:-1]

In [6]:
features

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
       'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40'],
      dtype='object')

In [7]:
perm_dict = {1:2, 2:1, 3:3, 4:6, 7:4, 6:5, 5:7,}

In [8]:
inv_perm = {v: k for k, v in perm_dict.items()}
inv_perm

{2: 1, 1: 2, 3: 3, 6: 4, 4: 7, 5: 6, 7: 5}

In [9]:
train['Cover_Type'].replace(perm_dict, inplace=True)


In [10]:
train = train[train.Cover_Type !=7]

In [11]:
target = train[['Cover_Type']].values
train.drop(['Cover_Type', 'Id'], axis=1, inplace=True)
test.drop(['Id', ], axis=1, inplace=True)

In [12]:
%%time
train_test = pd.concat([train, test], axis =0)
RS = RobustScaler()
RS.fit(train_test)
del train_test
gc.collect()
gc.collect()
train[features] = RS.transform(train)
test[features] = RS.transform(test)
del RS
gc.collect()
gc.collect()

CPU times: user 8.44 s, sys: 4.93 s, total: 13.4 s
Wall time: 13.4 s


0

In [13]:
train.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,0.474359,-0.441489,-0.545455,-0.721116,-0.243243,1.207143,-0.324324,0.370370,0.944444,2.675076,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.126068,0.313830,-0.818182,0.274900,-0.027027,1.207143,0.405405,0.592593,-0.666667,3.095566,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.297009,-0.585106,-0.636364,0.557769,0.081081,0.975974,-0.270270,0.370370,-0.092593,2.977829,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.117521,0.813830,-0.090909,-0.075697,-0.202703,1.053896,-0.297297,0.518519,0.259259,1.140673,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.130342,0.335106,-0.090909,0.219124,-0.121622,0.977273,0.351351,0.259259,0.222222,0.969419,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
xgtrain, xgval, y_train, y_val = train_test_split(train, target, test_size=0.2, random_state=42)

In [15]:
xgtrain['target'] = y_train
xgval['target'] = y_val

In [16]:
xgtrain.to_csv('../../TPS_2021/input/tabular-playground-series-dec-2021/xgtrain.csv', index=False)
xgval.to_csv('../../TPS_2021/input/tabular-playground-series-dec-2021/xgval.csv', index=False)